# 0. はじめに

物理学の実験においては、事前に測定値を予測することが可能な様々な理論式が知られています。理論式を検証するために実験を行うこともあれば、理論式に合わない実験結果を説明するために新たな理論が提唱されることもあります。ここではラザフォード散乱の式を例にとり、理論式を用いた予想と検証を行います。ラザフォード散乱による散乱事象の数$N_{exp}$は

$$
N_{exp}=B⋅t⋅N\frac{dσ}{d\Omega}(θ,ϕ)\cdotΔΩ
$$

と推測されます。$B$は単位時間に標的に入射するアルファ線数、$t$は測定時間、$N$は薄膜の単位面積あたりに存在する原子数、$dσ/d\Omega$は単位立体角当たりのラザフォード散乱の確率、$ΔΩ$はセンサーの見込み角です。これら推測に必要な各種パラメタを測定し、具体的な事象数の推測をこのノートでは目指します。

pythonやjupyter notebookの基本的な使い方に関しては、内田誠氏作成の[こちらのノートブック](https://github.com/ktodome/b3exp/blob/master/notebooks/01_python_basics.ipynb)などを参考にしてください。

まずは以下のセルを実行して、本ノートブックの操作に必要な設定を行います。

In [ ]:
# @title 0.1. 必要設定の読み込み

#スクリプト実行に必要なパッケージ。
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import poisson,norm

#用いる関数を定義
def gaussian(x, a, m, s):#ガウシアン
    return a*np.exp(-0.5*(x-m)**2/s**2)

def lin(x,a,b):
    return a*x+b

def dlin(x,a,b,dx,da,db):#直線フィッティングの誤差導出用関数
    dy=(x*da)**2
    dy+=(dx*a)**2
    dy+=db**2
    dy=np.sqrt(dy)
    #print("{:10.4f}, {:10.4f}, {:10.4f}".format((x*da),dx*a,db))
    return dy
def pathToTime(path):
    time_format='"%Y-%m-%d" "%H:%M:%S"'
    temp = np.genfromtxt(path, skip_header=1025,skip_footer=1,dtype=str)
    start_text=temp[0]+' '+temp[1]
    start_time=datetime.datetime.strptime(start_text, time_format)
    end_text=temp[2]+' '+temp[3]
    end_time=datetime.datetime.strptime(end_text, time_format)
    Time=(end_time-start_time).seconds
    return Time

In [ ]:
# @title 0.2. Googleドライブのマウント

#実験データなど、Googleドライブにあるファイルにアクセスしたい場合に実行する。自分のアカウントで要認証。
from google.colab import drive
drive.mount('/content/drive')

#ファイルを格納するディレクトリを指定、"b3exp_alpha"という名前のディレクトリをマイドライブ直下に作ることを推奨
basePath='/content/drive/MyDrive/b3exp_alpha/'

# 1.実験装置のキャリブレーション・測定

この実験ではα線源、金及びアルミ薄膜、SSDを用います。それぞれの性能の公称値は以下です。
*   線源：　2 MBq (1 Bq = 1 event/sec)
*   金箔：　1 μm, 19.3g/cm^3
*   アルミ箔：　10 μm, 2.69 g/cm^3

これらの公称値を参考にしつつ、装置のキャリブレーション(較正)と実際の厚さの測定を行います。

## 1.1. 装置のキャリブレーション

まずは信号のエネルギーとチャンネルの関係を調べます。パルス波の大きさを変えながら、チャンネルがどのように変化するか調べ、**パルス波の大きさが0の時のチャンネルを0 MeV相当のエネルギーとして求めます**。

パルス波の大きさを変化させながら、ADCチャンネルの中心値を読み取り1.1.1.を編集、随時1.1.2.を実行してプロットを確認しましょう。

In [ ]:
# @title 1.1.1. キャリブレーション用測定データ手入力

#まず目で読んだ値でプロットを作る。
ch=np.array([304,379,466,546,627,698,790])#読み取ったADCチャンネル値に置き換える
hight=np.array([200,250,300,350,400,450,500])#読み取ったパルス波の高さ[mV]に置き換える

dch=np.ones(ch.size)#x方向の誤差は1とする。
dhight=np.ones(ch.size)*10#y方向の誤差は10とする。

In [ ]:
# @title 1.1.2. 直線フィッティング(ch vs hight)

#中心チャンネルを結ぶよう直線フィッティングして、更に0 MeV相当のチャンネル数を算出。
par_line, cov_line = curve_fit(lin, ch, hight, sigma=dhight, absolute_sigma=True)#f(ch)=hightを直線(lin)でフィッティング

perr_line = np.sqrt(np.diag(cov_line))#誤差抽出

#テキスト出力
mat = np.vstack((par_line,perr_line)).T
df = pd.DataFrame(mat,index=("Slope", "Offset"), columns=("Estimate", "Std. error"))
print(df)

#プロット作成
x_func = np.arange(0,1024)
y_func = par_line[0]*x_func + par_line[1]

plt.errorbar(ch,hight,dhight,dch,fmt="o",label='ch vs hight data')
plt.plot(x_func,y_func,label='ch vs hight fit')
plt.xlabel('MCA channel [ch]', fontsize=18, fontfamily='serif')#x軸の名前
plt.ylabel('Signal Hight [mV]', fontsize=18, fontfamily='serif')#y軸の名前
plt.legend(loc='upper left')
plt.show()

切片から0 MeV相当のチャンネル数を導出できます。次を実行して計算しましょう。

In [ ]:
# @title 1.1.3. 0 MeV相当チャンネル導出
a=par_line[0]
b=par_line[1]
da=perr_line[0]
db=perr_line[1]

E0=(0-b)/a#E=a*ch+b -> ch=(E-b)/aで、E=0を求める
dE0=abs((0-b)/a*np.sqrt((da/a)**2+(db/b)**2))#b,aにつく誤差をそれぞれ評価
print("0 MeV= {:7.2f} +/- {:7.2f} ch".format(E0,dE0))

データが一通り出そろったら、もう少し丁寧な解析をしてみましょう。
1.1.1.では波高の誤差を大まかに決めましたが、実際に波高にはばらつきがあり、その範囲でおおよそ目標の波高に調整を行っているはずです。この波高のばらつきはMCA channelの幅として現れます。これを用いて誤差を丁寧に評価してみましょう。定量的に評価するために、ガウシアンでフィッティングを行います。
1.1.4.でこれまでのデータを取り込み、1.1.5.でそれぞれフィッティングを行います。
これらを行ってから再度1.1.2.を実行すると、より正確な傾きの見積もりができるようになります。

In [ ]:
# @title 1.1.4. キャリブレーション用測定データ解析
#パルス波ファイル、ファイル名と測定波高を任意の回数入れる
pulse_file_list=[]
pulse_file_list.append(['pulse500mV.csv',500])
pulse_file_list.append(['pulse450mV.csv',450])
pulse_file_list.append(['pulse400mV.csv',400])
pulse_file_list.append(['pulse350mV.csv',350])
pulse_file_list.append(['pulse300mV.csv',300])
pulse_file_list.append(['pulse250mV.csv',250])
pulse_file_list.append(['pulse200mV.csv',200])

pulse_list=np.empty((0,1024),int)
hight=[]
for file_i in pulse_file_list:#入力したデータ数分繰り返し
    y=np.genfromtxt(basePath+file_i[0], skip_footer=6)#1ヒストグラム取り出し
    y=[y]
    pulse_list=np.append(pulse_list,y,axis=0)
    hight=np.append(hight,file_i[1])


In [ ]:
# @title 1.1.5. キャリブレーション用ガウシアンフィッティング

#1.1.5のプロット用変数を定義
ch = []
dch = []
dhight=[]

pulses=np.zeros(1024)
dA=10

#プロット下地を作成
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)
x=np.arange(0,1024)#プロットするx軸のbinを設定
plt.xlabel('MCA channel [ch]', fontsize=18, fontfamily='serif')#x軸の名前
plt.ylabel('Event count', fontsize=18, fontfamily='serif')#y軸の名前
for hight_i,pulse_i in zip(hight,pulse_list):#入力したデータ数分繰り返し
    y=pulse_i
    dy=np.where(y > 0, np.sqrt(y), 1.0)#誤差は統計誤差の√nのみ,nが0なら0
    par_gaus, cov_gaus = curve_fit(gaussian, x, y, p0=(np.max(y), np.argmax(y), dA),sigma=dy, absolute_sigma=True)#適切な初期値を与えてフィッティング
    perr_gaus = np.sqrt(np.diag(cov_gaus))#分散共分散行列の対角成分取り出し=分散の取り出し
    #テキスト出力用成型
    mat = np.vstack((par_gaus,perr_gaus)).T
    df = pd.DataFrame(mat,index=("Constant", "Mean", "Sigma"), columns=("Estimate", "Std. error"))
    #テキスト出力
    print(df)
    #結果を記録
    ch=np.append(ch,float(par_gaus[1]))
    dch=np.append(dch,float(perr_gaus[1]))
    dhight=np.append(dhight,float(par_gaus[2])*a)

    print("{:3.2f}[mV]= {:.3f} +/- {:.3f}[ch] (dhight={:.3f} [mV])".format(hight_i,ch[-1],dch[-1],dhight[-1]))
    #フィット結果のプロットを追加
    y = gaussian(x, *par_gaus)
    _ = ax.plot(x, y, '-', label='fit{:5.1f}'.format(hight_i))
    _ = ax.hist(x, bins=1024, weights=y, range=(0, 1025), alpha=0.5, label='data{:5.1f}'.format(hight_i))
plt.legend(loc='upper right')#凡例表示

これでチャンネル数とパルス波の対応関係が得られました。再度1.1.2.と1.1.3.を行って、誤差がどの程度向上したか見てみましょう。

続いてアルファ線が、途中標的を通過せずにセンサーに届いた場合の信号のチャンネル数を調べます。アルファ線のエネルギーが4.5 MeVであることを用いて、キャリブレーションのスケールを再決定します。測定データを
```
NoTarget0deg60sec.csv
```
として保存してください。

続けて1.1.6.で実験データを読み込み、1.1.7.及び1.1.8.を実行して、キャリブレーションが適切に行われたか確認してください。

In [67]:
# @title 1.1.6. ターゲットなし測定データ入力
#強度測定ファイル、適宜書き換えること
calib_file='NoTarget0deg60sec.csv'

alpha_calib =np.genfromtxt(basePath+calib_file, skip_footer=6)
Tcalib=pathToTime(basePath+calib_file)

In [ ]:
# @title 1.1.7. ターゲットなしデータ用ガウシアンフィッティング

#プロット下地を作成
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)
_ = ax.hist(x, bins=1024, weights=alpha_calib, range=(0, 1024), alpha=0.5, label='source calibration data')#全データのヒストグラムを作成
plt.xlabel('MCA channel [ch]', fontsize=18, fontfamily='serif')#x軸の名前
plt.ylabel('Event count', fontsize=18, fontfamily='serif')#y軸の名前

dy=np.where(alpha_calib > 0, np.sqrt(alpha_calib), 1.0)#誤差は統計誤差の√nのみ,nが0なら0
par_calib, cov_calib = curve_fit(gaussian , x, alpha_calib, p0=(100, 800, 25), sigma=dy, absolute_sigma=True)#適切な初期値を与えてフィッティング
perr_calib = np.sqrt(np.diag(cov_calib))#分散共分散行列の対角成分取り出し=分散の取り出し

#テキスト出力
mat = np.vstack((par_calib,perr_calib)).T
df = pd.DataFrame(mat,index=("Constant", "Mean", "Sigma"), columns=("Estimate", "Std. error"))
print(df)

#フィット結果のプロットを追加
y = gaussian(x, *par_calib)
_ = ax.plot(x, y, '-', label='fit_calib')
plt.legend(loc='upper left')

In [ ]:
# @title 1.1.8. 4.5 MeV相当チャンネル導出とキャリブレーション結果確認
E4p5=par_calib[1]
dE4p5=perr_calib[1]
print("4.5 MeV= {:7.2f} +/- {:7.2f} ch".format(E4p5,dE4p5))

#4.5 MeVの高さが合うよう調整
scaling=a*E4p5 + b
par_line, cov_line = curve_fit(lin, ch, hight/scaling*4.5, sigma=dhight/scaling*4.5, absolute_sigma=True)#f(ch+/-dch)=Eを直線(lin)でフィッティング
perr_line = np.sqrt(np.diag(cov_line))#誤差抽出
a_ev=par_line[0]
b_ev=par_line[1]
da_ev=perr_line[0]
db_ev=perr_line[1]
y = a_ev*x + b_ev
print(a_ev,b_ev)

#キャリブレーションされた関数で、測定されたチャンネル数から予想されたエネルギーの値が得られるか確認
print("f(E4p5 [ch])= {:5.3f} +/- {:5.3f} [MeV]".format(lin(E4p5,a_ev,b_ev),dlin(E4p5,a_ev,b_ev,dE4p5,da_ev,db_ev)))
print("f(E0 [ch])= {:5.3f} +/- {:5.3f} [MeV]".format(lin(E0,a_ev,b_ev),dlin(E0,a_ev,b_ev,dE0,da_ev,db_ev)))

ch_eV=[E0,E4p5]
dch_eV=[dE0,dE4p5]
hight_eV=[lin(E0,a_ev,b_ev),lin(E4p5,a_ev,b_ev)]
dhight_eV=[dlin(E0,a_ev,b_ev,dE0,da_ev,db_ev),dlin(E4p5,a_ev,b_ev,dE4p5,da_ev,db_ev)]
fig = plt.figure(figsize=(6, 4))
plt.errorbar(ch_eV,hight_eV,dhight_eV,dch_eV,fmt="o",label='ch vs energy data (calibrated)')
plt.plot(x,y,label='ch vs energy fit')
plt.xlabel('MCA channel [ch]', fontsize=18, fontfamily='serif')#x軸の名前
plt.ylabel('Signal Energy [MeV]', fontsize=18, fontfamily='serif')#y軸の名前
plt.legend(loc='upper left')
plt.show()



f(ch)の出力は誤差の範囲で意図したエネルギーを示しているでしょうか？それが確認できたら、キャリブレーションは完了です。以下ではこの関数をチャンネル数からエネルギーへの変換関数として用います。

# 1.2. 線源の強度測定

続いて1.1.でキャリブレーションに用いたデータを用いて、ターゲットに入射する信号の強度を計算します。それには、線源からセンサーへの見込み角、線源からターゲットまでの見込み角を計算する必要があり、また線源の強度を推定する必要があります。

また、のちの計算のためにターゲットからセンサーへの見込み角も計算します。

一般に見込み角は
$$
\Delta\Omega=\frac{S}{r^2}
$$
で与えられます。

テキストで与えられている実験装置のジオメトリを用いて1.2.1.から1.2.3の計算を行います。

In [87]:
# @title 1.2.1. 線源からセンサーまでの見込み角

drs="値を入れる" #mm センサー手前のウィンドウ幅
Lrs="値を入れる" #mm 線源からセンサーまでの距離

if type(drs) is str:
  print("drsに数値を入力してください")
if type(Lrs) is str:
  print("Lrsに数値を入力してください")
if type(drs) is not str and type(Lrs) is not str:
  drs=float(drs)
  Lrs=float(Lrs)
  S2=np.pi*drs**2 #センサー手前のウィンドウ面積
  dOmega_RtoS=S2/Lrs**2 #線源からセンサーまでの見込み角
  print("線源からセンサーまでの見込み角={:10.3e} [sr]".format(dOmega_RtoS))

drsに数値を入力してください
Lrsに数値を入力してください


In [93]:
# @title 1.2.2. 線源からターゲットの見込み角

drt="値を入れる" #mm ターゲット手前のウィンドウ幅
Lrt="値を入れる" #mm 線源からターゲット手前のウィンドウまでの距離

if type(drt) is str:
  print("drtに数値を入力してください")
if type(Lrt) is str:
  print("Lrtに数値を入力してください")
if type(drt) is not str and type(Lrt) is not str:
  drt=float(drt)
  Lrt=float(Lrt)
  S1=np.pi*drt**2 #センサー手前のウィンドウ面積
  dOmega_RtoT=S1/Lrt**2 #線源からターゲットの見込み角
  #オプション課題：ターゲット手前のウィンドウの傾きを考慮するとどうなるか検討する
  print("線源からターゲットまでの見込み角={:10.3e} [sr]".format(dOmega_RtoT))



drtに数値を入力してください
Lrtに数値を入力してください


In [92]:
# @title 1.2.3. ターゲットからセンサーでの見込み角

Lts="値を入れる" #mm ターゲットからセンサーまでの距離
if type(Lts) is str:
  print("Ltsに数値を入力してください")
if type(Lts) is not str:
  dOmega_TtoS=S2/Lts**2 #ターゲットからセンサーまでの見込み角
  print("ターゲットからセンサーまでの見込み角={:10.3e} [sr]".format(dOmega_TtoS))

Ltsに数値を入力してください


ここまでで計算した見込み角を用いると、線源そのものの強度と、ターゲットに入射する信号の強度が計算できます。1.2.4.,1.2.5.で計算します。

In [ ]:
# @title 1.2.4. 線源強度

Ncalib=np.sum(alpha_calib)#キャリブレーションのデータ測定で得られたイベント数を全て足す

B0=Ncalib/Tcalib*4*np.pi/dOmega_RtoS#線源強度
dB0=np.sqrt(Ncalib)/Tcalib*4*np.pi/dOmega_RtoS#統計誤差
#オプション課題：不感時間を考慮するとどうなるか検討する
print("測定時間 {:.0f} [sec] 測定回数 {:8.3e} -> 線源強度={:10.3e} +/- {:7.1e} [Bq]".format(Tcalib,Ncalib,B0,dB0))


In [ ]:
# @title 1.2.5. ターゲットに入射する信号強度
B=B0*dOmega_RtoT/4/np.pi
dB=dB0*dOmega_RtoT/4/np.pi
print("ターゲットに入射する信号強度={:6.1f} +/- {:6.1f} [Bq]".format(B,dB))

公称値2 MBqに対して測定された強度は妥当か検討してください。

# 1.3. 薄膜の厚さ測定

続いて薄膜の厚さを測定します。テキストにあるように、薄膜を通過したアルファ線のエネルギーを評価することで、厚さを計算することができます。

実験の進行状況に応じて、以下のいずれかを行ってください。

*   シミュレーションデータで行う場合: 1.3.1.を実行
*   実験データ分布を全て用いる場合: 1.3.2.を実行

続けて1.3.3.及び1.3.4を実行して、アルファ線のエネルギーを導出します。

In [ ]:
# @title 1.3.2. 薄膜厚さ測定データ入力
#実データ用。ファイル名は適宜書き換えること
Aldeg0 = np.genfromtxt(basePath+'Al0deg60sec.csv', skip_footer=6)
Audeg0 = np.genfromtxt(basePath+'Au0deg60sec.csv', skip_footer=6)

In [ ]:
# @title 1.3.3. 薄膜厚さ測定用ガウシアンフィッティング

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(111)
_ = ax.hist(x, bins=1024, weights=alpha_calib, range=(0, 1024), alpha=0.5, label='Without target')
plt.xlabel('MCA channel [ch]', fontsize=18, fontfamily='serif')#x軸の名前
plt.ylabel('Event count', fontsize=18, fontfamily='serif')#y軸の名前

#Alに関する処理
dy=np.where(Aldeg0 > 0, np.sqrt(Aldeg0), 1.0)
par_Al, cov_Al = curve_fit(gaussian, x, Aldeg0, p0=(np.max(Aldeg0), np.argmax(Aldeg0), 30), sigma=dy)
perr_Al = np.sqrt(np.diag(cov_Al))
mat = np.vstack((par_Al,perr_Al)).T
df = pd.DataFrame(mat,index=("Constant", "Mean", "Sigma"), columns=("Estimate", "Std. error"))
print(df)
y = gaussian(x, *par_Al)
_ = ax.plot(x, y, '-', label='fitAl')
print("ch after Al: {:5.1f}+/-{:5.1f}".format(par_Al[1],perr_Al[1]))#中心値とその誤差を出力

#Auに関する処理
dy=np.where(Audeg0 > 0, np.sqrt(Audeg0), 1.0)
par_Au, cov_Au = curve_fit(gaussian, x, Audeg0, p0=(np.max(Audeg0), np.argmax(Audeg0), 25), sigma=dy)
perr_Au = np.sqrt(np.diag(cov_Au))
mat = np.vstack((par_Au,perr_Au)).T
df = pd.DataFrame(mat,index=("Constant", "Mean", "Sigma"), columns=("Estimate", "Std. error"))
print(df)
y = gaussian(x, *par_Au)
_ = ax.plot(x, y, '-', label='fitAu')
print("ch after Au: {:5.1f}+/-{:5.1f}".format(par_Au[1],perr_Au[1]))#中心値とその誤差を出力

_ = ax.hist(x, bins=1024, weights=Aldeg0, range=(0, 1024), alpha=0.5, label='Al calibration data')
_ = ax.hist(x, bins=1024, weights=Audeg0, range=(0, 1024), alpha=0.5, label='Au calibration data')
plt.legend(loc='upper left')


In [ ]:
# @title 1.3.4. 薄膜厚さ測定チャンネル数からエネルギーへ変換

#Alに関する変換
EAlreco=lin(par_Al[1],par_line[0],par_line[1])
dEAlreco=dlin(par_Al[1],par_line[0],par_line[1],perr_Al[1],perr_line[0],perr_line[1])
print("Al通過後のアルファ線のエネルギー : {:8.3f} +/- {:8.3f} MeV".format(EAlreco,dEAlreco))

#Auに関する変換
EAureco=lin(par_Au[1],par_line[0],par_line[1])
dEAureco=dlin(par_Au[1],par_line[0],par_line[1],perr_Au[1],perr_line[0],perr_line[1])
print("Au 通過後のアルファ線のエネルギー : {:8.3f} +/- {:8.3f} MeV".format(EAureco,dEAureco))

エネルギーはテキストにある理論曲線から厚さに変換することができます。

まずは1.3.5で理論曲線をフィットし、厚さとエネルギーの関係式を求めます。

更にその関係式から各薄膜の厚さを計算します。

In [ ]:
# @title 1.3.5.テキストにある理論値のフィッティング

#テキストにある理論値
Ealphatheo=[1.5, 2.0, ...]#アルファ線のエネルギー[MeV]
EAltheo=["値を入れる"]#Al中の飛程[um]
EAutheo=["値を入れる"]#Au中の飛程[um]

#Alに関するフィッティング
par_Al_line, cov_Al_line = curve_fit(lin, Ealphatheo, EAltheo)#アルファ線のエネルギー[MeV]->アルファ線が飛べるAlの厚さ[um]の式をフィッティング
perr_Al_line = np.sqrt(np.diag(cov_Al_line))
mat = np.vstack((par_Al_line,perr_Al_line)).T
df = pd.DataFrame(mat,index=("Slope", "Offset"), columns=("Estimate", "Std. error"))
print(df)

#Auに関するフィッティング
par_Au_line, cov_Au_line = curve_fit(lin, Ealphatheo, EAutheo)#アルファ線のエネルギー[MeV]->アルファ線が飛べるAlの厚さ[um]の式をフィッティング
perr_Au_line = np.sqrt(np.diag(cov_Au_line))
mat = np.vstack((par_Au_line,perr_Au_line)).T
df = pd.DataFrame(mat,index=("Slope", "Offset"), columns=("Estimate", "Std. error"))
print(df)

#プロット
plt.scatter(Ealphatheo,EAltheo,marker="o",label='Al theory')#理論値Al
plt.scatter(Ealphatheo,EAutheo,marker="o",label='Au theory')#理論値Au
plt.xlabel('Alpha ray energy [MeV]', fontsize=18, fontfamily='serif')#x軸の名前
plt.ylabel('Alpha ray range [μm]', fontsize=18, fontfamily='serif')#y軸の名前
x_func = np.arange(1.5,5.5)
y_funcAl = par_Al_line[0]*x_func + par_Al_line[1]
y_funcAu = par_Au_line[0]*x_func + par_Au_line[1]
plt.plot(x_func,y_funcAl,'b',label='Al fit')
plt.plot(x_func,y_funcAu,'r',label='Au fit')
plt.scatter(EAlreco,par_Al_line[0]*EAlreco+par_Al_line[1],marker="*",color='b',label='Al data')#測定値Al
plt.scatter(EAureco,par_Au_line[0]*EAureco+par_Au_line[1],marker="*",color='r',label='Au data')#測定値Au
plt.legend(loc='upper left')


In [ ]:
# @title 1.3.6.エネルギーから厚さへの変換

#厚さとエネルギーの関係式から、Al薄膜の厚さを求める
RAl=lin(EAlreco,par_Al_line[0],par_Al_line[1])#測定された、残りのエネルギーで飛べるAlの厚さ
dRAl=dlin(EAlreco,par_Al_line[0],par_Al_line[1],dEAlreco,perr_Al_line[0],perr_Al_line[1])#その誤差
dxAl=18.618-RAl#アルファ線が飛んできたAlの厚さ
print("Al残り飛程: {:10.2f} +/- {:10.2f} [um] -> 厚さdx={:10.2f} +/- {:10.2f} [um]".format(RAl,dRAl,dxAl,dRAl))

#以下Auに関して同様
RAu=lin(EAureco,par_Au_line[0],par_Au_line[1])
dRAu=dlin(EAureco,par_Au_line[0],par_Au_line[1],dEAureco,perr_Au_line[0],perr_Au_line[1])
dxAu=8.001-RAu
print("Au残り飛程: {:10.2f} +/- {:10.2f} [um] -> 厚さdx={:10.2f} +/- {:10.2f} [um]".format(RAu,dRAu,dxAu,dRAu))

求められた厚さを公称値と比較してください。

以上で装置のキャリブレーション・測定が完了しました。

# 2.測定値の推定

1.で得られた値を基に、大角度の散乱現象の測定計画を立てます。

まずは2.1.1.を実行して測定・計算された値を振り返りましょう。また厚さの値を用いて2.1.2.で単位面積当たりの原子数を計算しましょう。

In [ ]:
# @title 2.1.1.測定・計算された値の振り返り
print("ターゲットに入射する信号強度={:6.1f} +/- {:6.1f} [Bq]".format(B,dB))
print("ターゲットから線源までの見込み角={:10.3e} [sr]".format(dOmega_TtoS))
print("Al厚さdx={:10.2f} +/- {:10.2f} [um]".format(dxAl,dRAl))
print("Au厚さdx={:10.2f} +/- {:10.2f} [um]".format(dxAu,dRAu))
print("キャリブレーションを行った時間={:4.0f}[sec]".format(Tcalib))

In [ ]:
# @title 2.1.2.原子密度の計算
#計算用パラメタ
NA="値を入れる"#[cont/mol]
Al_weight="値を入れる"#[g/mol]
Au_weight="値を入れる"#[g/mol]
#Al密度計算
NAl=dxAl*1e-4*"値を入れる"*NA/Al_weight #dxAl[cm]*"値を入れる"[g/cm^3]*NA[count/mol]/weight[g/mol]
dNAl=dRAl*1e-4*"値を入れる"*NA/Al_weight
print("単位面積当たりアルミニウム原子数={:10.1e} [1/cm^2]={:10.1e} [1/fm^2]".format(NAl,NAl*1e-26))
#Au密度計算
NAu=dxAu*1e-4*"値を入れる"*NA/Au_weight
dNAu=dRAu*1e-4*"値を入れる"*NA/Au_weight
print("単位面積当たり金原子数={:10.1e} [1/cm^2]={:10.1e} [1/fm^2]".format(NAu,NAu*1e-26))
print("")

続いて2.1.3.で仮に実験計画のパラメタ設定をしてから、2.1.4.で各薄膜で散乱現象が起こる確率
$$
\frac{dσ}{d\Omega}(θ,ϕ)=(\frac{zZe^2}{4E_{α}})^2\frac{1}{\sin^4(θ/2)}
$$
を計算します。

In [ ]:
# @title 2.1.3.測定計画パラメタの設定
theta="値を入れる"
t="時間を入れる"*60*60#sec
print("測定時間{:10.0f}秒={:10.0f}分".format(t,t/60))

In [ ]:
# @title 2.1.4.散乱確率の見積もり
#計算用パラメタ
z = 2  # alpha は4He 2+
ZAl = 13  # Alの陽子数は13
ZAu = 79  # Auの陽子数は79
hbarc = 197.3
alpha = 1/137.
e2 = hbarc*alpha
E = 4.5

#Al散乱確率計算
AAl = (z*ZAl*e2/4/E)**2
dsigmadOmegaAl=AAl*(np.sin(np.radians(theta)/2))**-4
print("dsigma/dOmega(Al)={:10.1e}[fm^2/sr]".format(dsigmadOmegaAl))

#Au散乱確率計算
AAu = (z*ZAu*e2/4/E)**2
dsigmadOmegaAu=AAu*(np.sin(np.radians(theta)/2))**-4
print("dsigma/dOmega(Au)={:10.1e}[fm^2/sr]".format(dsigmadOmegaAu))

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
x=np.arange(5,90)#プロットするx軸のbinを設定
fAl=AAl*(np.sin(np.radians(x)/2))**-4
fAu=AAu*(np.sin(np.radians(x)/2))**-4
ax.plot(x, fAl, 'b-',label='Al theory')
ax.plot(x, fAu, 'r-',label='Au theory')
plt.scatter(theta,dsigmadOmegaAl,marker="o",color="b",label='Al expected')#理論値Al
plt.scatter(theta,dsigmadOmegaAu,marker="o",color="r",label='Au expected')#理論値Al
plt.yscale('log')  # ログスケール
ax.set_xlabel(r'$\theta$ (deg.)')
ax.set_ylabel(r'$d\sigma/d\Omega$ (fm$^2$)')
plt.legend(loc='upper right')

値は全て揃いました。0.で与えた関係式を用いて、予想される信号数を2.1.5.で計算します。

In [ ]:
# @title 2.1.5.散乱事象観測数の見積もり
N_expectedAl=B*t*NAl*1e-26*dsigmadOmegaAl*dOmega_TtoS
dN_expectedAl=B*t*NAl*1e-26*dsigmadOmegaAl*dOmega_TtoS*np.sqrt((dB/B)**2+(1/np.sqrt(NAl)**2)+(dNAl/NAl)**2)
print("推定測定回数(Al)={:10.1e}+/-{:10.1e}".format(N_expectedAl,dN_expectedAl))
N_expectedAu=B*t*NAu*1e-26*dsigmadOmegaAu*dOmega_TtoS
dN_expectedAu=B*t*NAu*1e-26*dsigmadOmegaAu*dOmega_TtoS*np.sqrt((dB/B)**2+(1/np.sqrt(NAu)**2)+(dNAu/NAu)**2)
print("推定測定回数(Au)={:10.1e}+/-{:10.1e}".format(N_expectedAu,dN_expectedAu))

推定された値はラザフォード散乱を"発見"するうえで十分だといえるでしょうか？2.1.3のパラメタを変更して様々なケースの推定を行ってみましょう。

オプション課題:小角度散乱の場合は多くの事象が期待されます。短時間の測定回数を高精度で予測できるでしょうか？寄与の大きな誤差は？